In [43]:
import pandas as pd
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import pydotplus
import optuna
import dtale

# read files
data_file = 'data/traindata.csv'
lable_file = 'data/trainlabel.txt'
judge_file = 'data/testdata.csv'
data_df = pd.read_csv(data_file)
judge_df = pd.read_csv(judge_file)
lable = []
with open(lable_file) as f:
    for line in f:
        lable.append(1 if line.strip() == '1' else 0)
data_df = data_df.join(pd.DataFrame(lable, columns=['label']))

# data cleansing
# mask = data_df.isin(['?'])
# unkown = mask.any(axis=1)
# data_df.drop(data_df[unkown].index, inplace=True)
# data_df = data_df.reset_index(drop=True)
# lable = [lable[i] for i in range(len(lable)) if not unkown[i]]
# weight = data_df['fnlwgt'].to_list()[:20001]
# data_df.drop('fnlwgt', axis=1, inplace=True)
# data_df = pd.get_dummies(data_df)
# features = data_df.columns.tolist()
dtale.show(data_df, open_browser=True)

In [138]:
dummies = pd.get_dummies(data_df.drop(columns=['sex', 'native.country', 'race']),
                         columns=['workclass', 'education', 'marital.status', 'occupation', 'relationship'])
train_df = dummies[:20001]
test_df = dummies[20000:]

rf = RandomForestClassifier(n_estimators=114, max_depth=68, min_samples_split=67, max_leaf_nodes=424)
rf.fit(train_df.drop(columns=['fnlwgt', 'label']), train_df['label'], train_df['fnlwgt'])
predictions = rf.predict(test_df.drop(columns=['fnlwgt', 'label']))
print(metrics.accuracy_score(predictions, test_df['label']))

# estimator = rf.estimators_[0]
# dot_data = export_graphviz(estimator,
#                 out_file= None,
#                 feature_names = features,
#                 class_names= ['True', 'False'],
#                 rounded = True,
#                 proportion = False,
#                 precision = 2,
#                 filled = True)
# graph = pydotplus.graph_from_dot_data(dot_data)
# graph.write_pdf('tree.pdf')

0.8592406876790831


In [136]:
def objective(trial:optuna.Trial):
    depth = trial.suggest_int('depth', 7, 86)
    split = trial.suggest_int('split', 49, 2333)
    leaves = trial.suggest_int('leaves', 360, 4090)
    rf = RandomForestClassifier(n_estimators=44, max_depth=depth, min_samples_split=split, max_leaf_nodes=leaves)
    # {'depth': 41, 'split': 55, 'leaves': 417}
    # {'depth': 55, 'split': 14, 'leaves': 737}
    # {'depth': 49, 'split': 107, 'leaves': 465}
    rf.fit(train_df.drop(columns=['fnlwgt', 'label']), train_df['label'], train_df['fnlwgt'])
    predictions = rf.predict(test_df.drop(columns=['fnlwgt', 'label']))
    acc = metrics.accuracy_score(predictions, test_df['label'])
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=233, n_jobs=-1)
study.best_params, study.best_value

[I 2023-06-09 00:13:55,557] A new study created in memory with name: no-name-ef27da31-afe2-4ffe-89d4-78ac5bc525c1
[I 2023-06-09 00:13:57,731] Trial 12 finished with value: 0.8363180515759312 and parameters: {'depth': 7, 'split': 2125, 'leaves': 3312}. Best is trial 12 with value: 0.8363180515759312.
[I 2023-06-09 00:13:57,751] Trial 13 finished with value: 0.836676217765043 and parameters: {'depth': 85, 'split': 2128, 'leaves': 1526}. Best is trial 13 with value: 0.836676217765043.
[I 2023-06-09 00:13:57,798] Trial 6 finished with value: 0.8459885386819485 and parameters: {'depth': 66, 'split': 1584, 'leaves': 2708}. Best is trial 6 with value: 0.8459885386819485.
[I 2023-06-09 00:13:57,855] Trial 2 finished with value: 0.836676217765043 and parameters: {'depth': 67, 'split': 1886, 'leaves': 2947}. Best is trial 6 with value: 0.8459885386819485.
[I 2023-06-09 00:13:57,886] Trial 3 finished with value: 0.8474212034383954 and parameters: {'depth': 82, 'split': 1876, 'leaves': 1323}. Best

({'depth': 45, 'split': 76, 'leaves': 2283}, 0.8603151862464183)

In [133]:
dummies = pd.get_dummies(data_df.drop(columns=['native.country']),
                         columns=['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race'])
train_df = dummies[:20001]
test_df = dummies[20000:]
group = train_df.groupby('sex')
classifiers = {}
for country, df in group:
    classifiers[country] = RandomForestClassifier(n_estimators=98, max_depth=21, min_samples_split=219, max_leaf_nodes=1039)
    classifiers[country].fit(df.drop(columns=['sex', 'fnlwgt', 'label']), df['label'], df['fnlwgt'])
    
group = test_df.groupby('sex')
predictions_all, labels_all = [], []
for country, df in group:
    predictions = classifiers[country].predict(df.drop(columns=['sex', 'fnlwgt', 'label']))
    predictions_all += list(predictions)
    labels_all += df['label'].tolist()
    
print(metrics.accuracy_score(predictions_all, labels_all))

0.8542263610315186


In [177]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

dummies = pd.get_dummies(data_df.drop(columns=['sex', 'native.country', 'race', 'marital.status']),
                         columns=['workclass', 'education', 'occupation', 'relationship'])
train_df = dummies[:20001]
test_df = dummies[20000:]

bnb = BernoulliNB()
bnb.fit(train_df.drop(columns=['fnlwgt', 'label']), train_df['label'], train_df['fnlwgt'])
predictions = bnb.predict(test_df.drop(columns=['fnlwgt', 'label']))
print(metrics.accuracy_score(predictions, test_df['label']))

canb = CategoricalNB()
canb.fit(train_df.drop(columns=['fnlwgt', 'label']), train_df['label'], train_df['fnlwgt'])
predictions = canb.predict(test_df.drop(columns=['fnlwgt', 'label']))
print(metrics.accuracy_score(predictions, test_df['label']))

conb = ComplementNB()
conb.fit(train_df.drop(columns=['fnlwgt', 'label']), train_df['label'], train_df['fnlwgt'])
predictions = conb.predict(test_df.drop(columns=['fnlwgt', 'label']))
print(metrics.accuracy_score(predictions, test_df['label']))

gnb = GaussianNB()
gnb.fit(train_df.drop(columns=['fnlwgt', 'label']), train_df['label'], train_df['fnlwgt'])
predictions = gnb.predict(test_df.drop(columns=['fnlwgt', 'label']))
print(metrics.accuracy_score(predictions, test_df['label']))

mnb = MultinomialNB()
mnb.fit(train_df.drop(columns=['fnlwgt', 'label']), train_df['label'], train_df['fnlwgt'])
predictions = mnb.predict(test_df.drop(columns=['fnlwgt', 'label']))
print(metrics.accuracy_score(predictions, test_df['label']))

0.8244985673352435
0.8420487106017192
0.7818767908309455
0.8291547277936963
0.7818767908309455
